# 1. 모델 구현하기
* https://www.tensorflow.org/guide/keras/sequential_model
```
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)
```


# 2. 알아야 할 용어들
* CNN
   - CNN은 전통적인 뉴럴 네트워크 앞에 여러 계층의 컨볼루셔널 계층을 붙인 형태
   - 컨볼루셔널 계층을 통해서 입력받은 이미지에 대한 특징(feature)를 추출하게 되고, 추출한 특징을 기반으로
     기준의 뉴럴 네트워크를 이용하여 분류
     
     
* 컨볼루셔널 레이어(Convolutional Layer)
  - 입력 데이터로부터 특징을 추출하는 역할
  - 특징을 추출하는 필터(Filter)를 사용함
  - 필터의 값을 비선형 값으로 바꿔주는 activation 함수를 사용


* 필터(Filter)
  - 특징이 데이터에 있는지, 없는지 검출하는 함수
  - 필터는 구현에서 행렬로 정의
  - 입력받은 이미지 모두 행렬로 변환
  - 입력받은 데이터에서 그 특징을 가지고 있으면 결과값이 큰 값으로 나온다.(어떠한 값이 나올지는 알 수 없다.)
  - 특징을 가지고 있지 않다면 0에 가까운 값이 반환된다.
  
  
* strides
    - 필터를 적용하는 간격(예: 우측으로 한칸씩 아래로 한칸씩 적용)
    - 필터를 적용해서 얻어낸 결과 
    - Feature map 또는 activation map이라 부름
    
    
* padding
    - CNN 너트워크 특징 중 여러 단계에 걸쳐서 계속 필터를 연속적으로 적용하는데 필터 적용 후
      결과값이 작아지게 되면 처음에 비해 특징이 많이 유실될 수 있음
    - 충분히 특징이 추출되기 전에 결과 값이 작아지면 특징이 유실되므로 이를 방지하기 위해 사용하는 기법
    - 입력값 주위로 0값을 넣어서 입력 값의 크기를 인위적으로 키워 결과값이 작아지는 것을 방지
    
    
* activation
  - 필터를 통해서 추출한 값이 들어가 있는 큰 값, 0에 가까운 값들을 비선형 값으로 바꿔주는 과정이 필요함
  - 시그모이드 함수: 뉴럴 네트워크에서 신경망이 깊어질수록 학습이 어렵기 때문에 전체 레이어를 한 번 계산한 후 그 계산 값을 재활용하거나 다시 계산하는 Back propagation 방법을 사용하는데, sigmoid 함수(0에 가까우면 False, 1에 가까우면 True를 반환해주는 함수)는 propagation이 제대로 작동하지 않아 ReLu를 사용하게 됨
  
  
* 풀링(Pooling)
  - max pooling : activation map을 M * N의 크기로 잘라낸 후 그 안에서 가장 큰 값을 뽑아내는 방법
  - 큰 값이 다른 특징들을 대표한다는 개념
  - 전체 데이터의 사이즈가 줄어들기 때문에, 연산에 들어가는 리소스가 작아짐
  - 데이터의 크기를 줄이면서 소실이 발생하기 때문에, 오버피팅을 방지
  
  
* BatchNormalization
  - 학습하는 과정 자체를 전체적으로 안정화시키고, 학습 속도를 가속시킬 수 있는 근본적인 방법을 제공하는 함수
  - 모델 업데이트
  
  
* Dropout
  - overfitting을 해소하기 위한 방법
  - 모델이 학습 데이터에 대해 지나치게 훈련되어 실험데이터(test data)에 대해 결과가 좋지 못할 경우 동작함
 

# 3. 모델 구현하기

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
import os

### 3-1. 기존 클래스의 옵션 변경하기

In [4]:
# 기존에 존재하는 클래스 사용하기(옵션 변경하기)
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # input
            layers.Input(input_shape),
            
            # 1번쨰 layer
            # Convolutional layer
            # LAYERS.Conv2D(컨볼루션 필터의 수, 컨볼루션 커널의 행,열,strides,activation,padding)
            # 컨볼루션 필터의 수: 합성곱 연산에서 사용되는 필터는 이미지에서 특징을 분리해내는 기능을 수행함
            # 필터의 종류(개수), 출력 공간의 차원을 결정함
            # 컨볼루션 커널(kernal_size)의 행/열: 합성곱에 사용되는 필터의 크기
            
            # padding  ='same': 텐서플로우가 자동으로 패딩을 삽입해 입력값과 출력값의 크기를 맞춰줌
            # padding = 'valid': 텐서플로우가 자동으로 패딩을 적용하지 않고, 필터를 적용하기 때문에 출력값의 크기가 작아짐
            layers.Conv2D(64,3,strides = 1,activation = "relu",padding = 'same'), # 이미지를 위한 레이어
            layers.Conv2D(64,3,strides = 1,activation = "relu",padding = 'same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            
            # 2번쨰 layer
            # 비슷한 구조의 레이어를 두 번 쌓는 이유: 학습의 효율을 높이기 위해서
            layers.Conv2D(128,3,strides = 1,activation = "relu",padding = 'same'),
            layers.Conv2D(128,3,strides = 1,activation = "relu",padding = 'same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # Classifier
            layers.GlobalMaxPool2D(),
            layers.Dense(128,activation = "relu"),
            layers.Dense(1,activation = "sigmoid")
        ]
    )
    return model

In [5]:
input_shape = (256,256,3)
model = get_sequential_model(input_shape)

model.summary() # 모델의 성능 확인

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     7

### 3-2. 함수 형태로 layering 진행하기

In [7]:
# 함수 형태로 layering하기
def get_functional_model(input_shape):
    inputs = keras.Input(input_shape)
    
    # 1st Conv2D
    x = layers.Conv2D(64,3,strides = 1,activation = "relu",padding = 'same')(inputs)
    x = layers.Conv2D(64,3,strides = 1,activation = "relu",padding = 'same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # 2nd Conv2D
    x = layers.Conv2D(128,3,strides = 1,activation = "relu",padding = 'same')(x)
    x = layers.Conv2D(128,3,strides = 1,activation = "relu",padding = 'same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Classifier
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dense(128,activation = "relu")(x)
    outputs = layers.Dense(1,activation = "sigmoid")(x)
    
    model = keras.Model(inputs,outputs)
    
    return model

In [8]:
input_shape = (256,256,3)
model = get_functional_model(input_shape)

model.summary() # 모델의 성능 확인

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 conv2d_5 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 128, 128, 64)      0   

### 3-3. 직접 클래스 만들기

In [11]:
# 클래스로 구현하기
class SimpleCNN(keras.Model):
    def __init__(self): # CNN 클래스 상속받기 -> 생성자 호출
        super(SimpleCNN,self).__init__()
        
        self.conv_block_1 = keras.Sequential(
            [
                layers.Conv2D(64,3,strides = 1,activation = "relu",padding = "same"),
                layers.Conv2D(64,3,strides = 1,activation = "relu",padding = "same"),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.5)
            ], name = 'conv_block_1'
        )
        
        self.conv_block_2 = keras.Sequential(
            [
                layers.Conv2D(128,3,strides = 1,activation = "relu",padding = "same"),
                layers.Conv2D(128,3,strides = 1,activation = "relu",padding = "same"),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.3)
            ], name = 'conv_block_2'
        )
        
        self.classifier = keras.Sequential(
        [
            layers.GlobalMaxPool2D(),
            layers.Dense(128,activation = "relu"),
            layers.Dense(1,activation = 'sigmoid')
        ], name = 'classifier'
    )
    
    def call(self,input_tensor,training = False): # callback 함수
        x = self.conv_block_1(input_tensor)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        
        return x

In [12]:
input_shape = (None,256,256,3) # 제일 앞의 parameter는 batch_size
model = SimpleCNN()
model.build(input_shape)

model.summary()

Model: "simple_cnn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block_1 (Sequential)   (None, 128, 128, 64)      38976     
                                                                 
 conv_block_2 (Sequential)   (None, 64, 64, 128)       221952    
                                                                 
 classifier (Sequential)     (None, 1)                 16641     
                                                                 
Total params: 277,569
Trainable params: 277,185
Non-trainable params: 384
_________________________________________________________________


In [13]:
# binary_crossentropy
# 클래스가 두 개인 이진 분류 문제에서 사용함
# label이 0 또는 1 값을 가질 때 사용
# 모델의 마지막 레이어의 활성화 함수 -> 시그모이드 함수
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = 'accuracy'
)